In [1]:
import datetime
import numpy as np
import talib as ta
import alpaca_trade_api as tradeapi
import time
import sys
sys.path.insert(0, '../')



In [2]:
from config_paper import key_id, secret_key
from cross_signal import *

In [3]:
#The url at the end is what makes a paper trading bot
base_url='https://paper-api.alpaca.markets'
api = tradeapi.REST(key_id, secret_key,base_url )

In [4]:
startdate = datetime.datetime.today().strftime("%Y-%m-%d")
startdate

'2020-04-27'

In [5]:
#make sure to import these packages
from pandas import DataFrame
import talib.abstract as tab
#abstract automatically extracts the columns and periods from the data itself, no input required



#feel free to add or subtract more indicators for any other strategy you want to use
def populateindicators(dataframe) -> DataFrame:
        #exponential moving averages
        dataframe['mfi'] = ta.MFI(dataframe['high'],dataframe['low'],dataframe['close'], np.asarray(dataframe['volume'], dtype='float'), timeperiod=14)
        
        dataframe['minusdi'] = tab.MINUS_DI(dataframe, timeperiod=14)
        dataframe['plusdi'] = tab.PLUS_DI(dataframe, timeperiod=14)
        
        dataframe['rsi'] = tab.RSI(dataframe, timeperiod=20, price='close')
        
        dataframe['adx'] = tab.ADX(dataframe)

        # required for graphing

        return dataframe

In [6]:
account = api.get_account()
print(account.status)
account.buying_power

ACTIVE


'374896.48'

In [23]:
api.get_portfolio_history()

PortfolioHistory({   'base_value': 100000,
    'equity': [   100000,
                  100000,
                  100000,
                  100000,
                  100000,
                  100000,
                  100000,
                  100000,
                  100000,
                  100000,
                  100000,
                  100000,
                  100000,
                  100000,
                  100000,
                  100000,
                  100000,
                  100000,
                  100000,
                  100000,
                  99059.87],
    'profit_loss': [   0,
                       0,
                       0,
                       0,
                       0,
                       0,
                       0,
                       0,
                       0,
                       0,
                       0,
                       0,
                       0,
                       0,
                       0,
                  

In [7]:
with open('SP500_2020Q2SYM.txt', 'r') as f:
    symbols= f.read().split('\n')

In [8]:
barTimeframe = "1Min" # 1Min, 5Min, 15Min, 1H, 1D
assetsToTrade = [x.upper() for x in symbols]
positionSizing = 0.25

# Tracks position in list of symbols to download 
assetListLen = len(assetsToTrade)


In [9]:
returneddata = api.get_barset('FLS',barTimeframe,limit=150).df
returneddata2=api.polygon.historic_agg_v2( 'EA', 1, 'minute',_from='2020-04-23', to=startdate,).df
#data_indic=populateindicators(returneddata['AMP'])


In [10]:
data=populateindicators(returneddata2)

In [30]:
try:
    openPosition = api.get_position('UAL')
    print(openPosition.qty)
except:
    print('oops')

587


In [12]:
targetPositionSize , cashBalance

(161, '74799.59')

In [27]:
api.submit_order('AAPL',161,"sell","market","day")

Order({   'asset_class': 'us_equity',
    'asset_id': 'b0b6dd9d-8b9b-48a9-ba46-b9d54906e415',
    'canceled_at': None,
    'client_order_id': '42c54a50-8bdf-4d3a-a2ba-68a352cf5ef9',
    'created_at': '2020-04-27T19:51:51.099922Z',
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'id': '2084d809-91a7-4a92-a37d-0862b62af74b',
    'legs': None,
    'limit_price': None,
    'order_class': '',
    'order_type': 'market',
    'qty': '161',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': 'sell',
    'status': 'accepted',
    'stop_price': None,
    'submitted_at': '2020-04-27T19:51:51.089656Z',
    'symbol': 'AAPL',
    'time_in_force': 'day',
    'type': 'market',
    'updated_at': '2020-04-27T19:51:51.099922Z'})

In [ ]:
for minu in range(500) :
        
    for i in range(assetListLen) :
        try:
            symbol = assetsToTrade[i]

            returned_data = api.polygon.historic_agg_v2(symbol, 1, 'minute',_from='2020-04-20', to=startdate,).df
            #api.get_barset(symbol,barTimeframe,limit=150).df


            # Calculated trading indicators
            #populate exponential moving averages
            data=populateindicators(returned_data) 


             # Create the buy signal here
            if data['adx'][-1] > 22 and \
            data['mfi'][-1] < 25 and \
            data['rsi'][-1] < 35 and \
            data['minusdi'][-1] > 25 and \
            data['minusdi'][-1] < data['plusdi'][-1]:

                try:
                    openPosition = api.get_position(symbol)
                    print(openPosition.qty)
                except:
                    cashBalance = account.cash

                    targetPositionSize = int(float(cashBalance) / (data['close'][-1] / positionSizing)) # Calculates required position size

                    returned = api.submit_order(symbol,targetPositionSize,"buy","market","day") # Market order to open position
                    print(returned)


            elif data['adx'][-1] > 22 and \
            data['mfi'][-1] > 63 and \
            data['rsi'][-1] > 74 and \
            data['plusdi'][-1] > 25 and \
            data['minusdi'][-1] < data['plusdi'][-1]:
                # Closes position  SMA50
                openPosition = api.get_position(symbol)

                returned = api.submit_order(symbol,int(openPosition.qty),"sell","market","day") # Market order to fully close position
                print(returned)
        
        except:
                print(symbol)
                
        #interval should be an integer, the number of seconds to wait
    time.sleep(20)

MMM
MO
BRKB
BFB
FTV
ITW
JCI
KSS
M
MKTK
MYL
NUE
ORCL
PWR
RTN
UTX

ANET
BRKB
BFB
CE
DD
FTV
JCI
MKTK
NUE
RTN
TSCO
UTX

MMM
APA
BRKB
BFB
DISCA
JCI
M
MKTK
